In [3]:
%pip install pandas numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt pandas_ta

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=a2bbd7ee167a3a07d32c6f9e0dfc74c665bba11ccf3ef9f0e9bfa06e63970927
  Stored in directory: /home/codespace/.cache/pip/wheels/fd/ed/18/2a12fd1b7906c63efca6accb351929f2c7f6bbc674e1c0ba5d
Successfully built pandas_ta
Note: you may need to restart the kernel to use updated packages.


In [4]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [7]:
print(pd.Timestamp.today().date())

2024-09-22


In [ ]:
# Load and manipulate most recent S&P 500 companies from Wikipedia (not survivorship-bias free)

# Loads several dfs, of which we only want the first
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Some symbols contain dots, replace with dash to prevent issues w yfinance
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Get the unique symbols (added robustness)
symbols_list = sp500['Symbol'].unique().tolist()

# Set end date to current date
end_date = pd.Timestamp.today().date()

# Alternatively use a fixed date
end_date = '2024-01-01'

# Set start date to be 8 years prior to end date
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# Download data from yfinance using tickers and date range (Time consuming)
# Stack method creates multi-index which is more convenient (reduces number of columns)
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

# Assign new names for multi-index df
df.index.names = ['date', 'ticker']

# Assign column headers to strings (easier for manipulation later)
df.columns = df.columns.str.lower()

df